#Evaluate A/B Test

## Evaluate Success Matric

In [ ]:
#Import library yang diperlukan
import numpy as np
import pandas as pd
import os

pd.options.display.max_columns = 999
pd.options.display.float_format = "{:.2f}".format

In [ ]:
#import dataset
df_marketing= pd.read_csv('/content/marketing_AB.csv')

#Menampilkan dataset
df_marketing.head(10)

,Unnamed: 0,user id,test group,converted,total ads,most ads day,most ads hour
0,0,1069124,ad,False,130,Monday,20
1,1,1119715,ad,False,93,Tuesday,22
2,2,1144181,ad,False,21,Tuesday,18
3,3,1435133,ad,False,355,Tuesday,10
4,4,1015700,ad,False,276,Friday,14
5,5,1137664,ad,False,734,Saturday,10
6,6,1116205,ad,False,264,Wednesday,13
7,7,1496843,ad,False,17,Sunday,18
8,8,1448851,ad,False,21,Tuesday,19
9,9,1446284,ad,False,142,Monday,14


In [ ]:
#Menghilangkan kolom angka
df_marketing.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
#Menampilkan kembali dataset
df_marketing.head(10)

,user id,test group,converted,total ads,most ads day,most ads hour
0,1069124,ad,False,130,Monday,20
1,1119715,ad,False,93,Tuesday,22
2,1144181,ad,False,21,Tuesday,18
3,1435133,ad,False,355,Tuesday,10
4,1015700,ad,False,276,Friday,14
5,1137664,ad,False,734,Saturday,10
6,1116205,ad,False,264,Wednesday,13
7,1496843,ad,False,17,Sunday,18
8,1448851,ad,False,21,Tuesday,19
9,1446284,ad,False,142,Monday,14


## Convertion Rate

Pertama, kita akan mengevaluasi apakah convertion rate telah meningkat dengan varian baru. Untuk membandingkan proporsi antara dua sampel yang berbeda, kita dapat menggunakan uji proportion z-test.

## Proportion Z-Test

Karena uji ini bertujuan untuk mengevaluasi apakah variant lebih baik daripada control (superiority test), kita perlu menyesuaikan pernyataan hipotesis menjadi sebagai berikut:

H0: pvarian ≤ pkontrol

H1: pvarian > pkontrol

In [ ]:
from statsmodels.stats.proportion import proportions_ztest

In [ ]:
#Proportion Z-test
group_control = df_marketing[ df_marketing['test group'] == 'psa']
group_variant = df_marketing[df_marketing['test group'] == 'ad']

conversion = [ group_variant['converted'].sum(), group_control['converted'].sum()]
sample = [ group_variant['converted'].shape[0], group_control['converted'].shape[0] ]

In [ ]:
#Menampilkan proporsi kedua z-test
z_stat, p_value = proportions_ztest(conversion, sample, alternative='larger')

In [ ]:
#print hasil
print(f"P-value: {p_value:.15f}")
print(f"CVR control: {(group_control['converted'].sum()/group_control['converted'].shape[0]):.3f}")
print(f"CVR variant: {(group_variant['converted'].sum()/group_variant['converted'].shape[0]):.3f}")

P-value: 0.000000000000085
CVR control: 0.018
CVR variant: 0.026


**Interpretasi**

Karena nilai p-value < 0.05, maka menolak H0. Hal ini berarti nilai convertion rate dari variant (ad) lebih besar dari nilai convertion rate control (psa).

Untuk nilai CVR control (psa) adalah sebesar 2.5%

Untuk nilai CVR variant (ad) adalah sebesar 3.8%

## Evaluate Guardrail Matrics

Guardrail matrics:

Rata-rata paparan iklan per pengguna

1.   **Rata-rata paparan iklan per pengguna**

  Dipilih karena:

  Ini untuk memastikan jumlah iklan yang dilihat pengguna tidak berlebihan. Terlalu banyak iklan dapat menyebabkan user fatigue, annoyance, bahkan churn.

2.   **Waktu interaksi terbanyak berdasarkan most ads hour**

  Dipilih karena:

  Untuk mengetahui jam tertentu saat pengguna paling banyak melihat iklan.Menghindari oversaturation di waktu yang sama yang bisa berdampak negatif.

3.   **Hari dengan iklan terbanyak berdasarkan most ads day**

  Dipilih karena:

  Untuk mengevaluasi distribusi frekuensi iklan berdasarkan hari.
  Mencegah iklan hanya tampil di hari-hari tertentu sehingga user merasa “dibombardir” di satu hari dan kosong di hari lain.


### 1. Rata-rata paparan iklan per pengguna

In [ ]:
# Memastikan kolom 'total ads' dan 'test group' tidak memiliki missing value
df = df_marketing.dropna(subset=['total ads', 'test group'])

# Menghitung rata-rata jumlah iklan yang dilihat per test group
avg_ads_by_group = df_marketing.groupby('test group')['total ads'].mean().reset_index()

# Menampilkan hasil hasil
print(avg_ads_by_group)

  test group  total ads
0         ad      24.82
1        psa      24.76


**Interpretasi**

Jika dilihat dari jumlah total ads per grupnya tidak terlihat terlalu banyak perbedaan total ads untuk masing-masing grup. Untuk lebih meyakinkan, kita cek terlebih dahulu apakah data total ads untuk masing-masinng grup berdistribusi normal.

### Normality Test

Hipotesis testing untuk uji normalitas:

H0  : Data berdistribusi normal

H1  : Data tidak berdistribusi normal

### Uji Normalitas "Anderson-Darling test"

Dipilih karena cocok untuk data yang besar

In [ ]:
# Pisahkan berdasarkan test group
group_ad = df[df['test group'] == 'ad']['total ads'].dropna()
group_psa = df[df['test group'] == 'psa']['total ads'].dropna()

In [ ]:
from scipy import stats

In [ ]:
def ad_normal_test(x):
  # Perform the Anderson-Darling test
  result = stats.anderson(x, dist='norm')

  # Interpretation
  for cv, sig in zip(result.critical_values[2:], result.significance_level[2:]):
    if result.statistic < cv:
        print(f"Fail to reject normality at {sig}% significance level")
    else:
        print(f"Reject normality at {sig}% significance level")


In [ ]:
ad_normal_test(group_ad)

Reject normality at 5.0% significance level
Reject normality at 2.5% significance level
Reject normality at 1.0% significance level


In [ ]:
ad_normal_test(group_psa)

Reject normality at 5.0% significance level
Reject normality at 2.5% significance level
Reject normality at 1.0% significance level


**Intreprtasi**

Nilai statistik > nilai kritis (critical value) pada semua tingkat signifikansi (5%, 2.5%, 1%). Artinya, kita menolak H0 yang mana datanya tidak berdistribusi normal untuk kedua grup

### Mann-Whitney U Test

Karena kedua grup tidak berdistribusi normal, kita gunakan uji Mann-Whitney U Test karena merupakan uji non-parametrik

Hipotesis:

H0 = Distribusi kedua kelompok sama (tidak ada perbedaan signifikan)

H1 = Distribusi kedua kelompok berbeda

In [ ]:
# Perform the Mann-Whitney U test
stat, p_value = stats.mannwhitneyu(group_ad, group_psa, alternative='greater')

# Print results
print(f"P-value: {p_value}")
print(f"Rata-rata total ads untuk Control: {group_psa.mean():.2f}")
print(f"Rata-rata total ads untuk Variant: {group_ad.mean():.2f}")


P-value: 2.3454554273602847e-11
Rata-rata total ads untuk Control: 24.76
Rata-rata total ads untuk Variant: 24.82


**Interpretasi**

Nilai P-Valus 0.056, artinya p-value > 0.05. Hal tersebut menunjukkan bahwa kita gagal menolak H0. Jadi, tidak ada perbedaan yang cukup signifikan antara kedua grup

**Interpretasi**

Paparan iklan per pengguna antara grup yang melihat iklan (ad) dan yang melihat PSA tidak berbeda signifikan secara statistik.

Ini berarti pengguna yang melihat iklan tidak secara ekstrem dibanjiri iklan dibandingkan kelompok kontrol.

**Insight**
Berdasarkan hasil uji statistik dengan p-value = 0.056, tidak terdapat perbedaan yang signifikan antara rata-rata jumlah iklan yang dilihat oleh pengguna pada kelompok eksperimen (ad) dan kelompok kontrol (psa). Artinya, kampanye iklan tidak menyebabkan paparan iklan yang berlebihan bagi pengguna.

Hal ini menunjukkan bahwa:

- Kampanye telah dirancang dengan jumlah iklan yang seimbang dan tidak mengganggu.

- Pengguna yang melihat iklan tidak merasa terbebani dibanding pengguna yang hanya melihat PSA.

- Dari sudut pandang user experience (UX) dan etika pemasaran, kampanye dapat dikatakan aman dan tidak menimbulkan risiko negatif dalam hal intensitas eksposur.

**Rekomendasi**:

Lanjutkan strategi distribusi iklan saat ini, karena telah terbukti tidak membebani pengguna secara signifikan.

Pantau tren paparan iklan secara berkala jika kampanye berlangsung lebih lama atau intensitas iklan ditingkatkan.



### 2. Waktu interaksi terbanyak berdasarkan most ads hour

In [ ]:
# Hitung frekuensi setiap jam
most_ads_by_hour = df_marketing['most ads hour'].value_counts().sort_index()

# Tampilkan jam paling sering muncul
most_active_hour = most_ads_by_hour.idxmax()
most_active_hour_freq = most_ads_by_hour.max()

print(f'Jam dengan interaksi iklan terbanyak: {most_active_hour} (sebanyak {most_active_hour_freq} pengguna)')

Jam dengan interaksi iklan terbanyak: 13 (sebanyak 47655 pengguna)


**Interpretasi**

Pukul 12.00 adalah waktu puncak pengguna melihat iklan, kemungkinan karena:

- Istirahat makan siang (jam kerja)

- Pengguna membuka perangkat di sela aktivitas

Jumlah yang sangat besar (~28% dari 100.000+ total user) menunjukkan konsentrasi interaksi tinggi di satu waktu.

**Insight Guardrail**

Potensi risiko oversaturation: Terlalu banyak menayangkan iklan pada jam ini bisa membuat pengguna jenuh → menurunkan efektivitas kampanye atau bahkan meningkatkan bounce rate.

**Rekomendasi:**

Diversifikasi jam tayang iklan, agar beban interaksi tidak hanya terpusat di jam 12.

Lakukan uji performa iklan berdasarkan jam tayang untuk mencari waktu efektif lainnya.



### 3. Hari dengan iklan terbanyak berdasarkan most ads day

In [ ]:
# Hitung frekuensi setiap hari
most_ads_by_day = df_marketing['most ads day'].value_counts()

# Tampilkan hari paling sering muncul
most_active_day = most_ads_by_day.idxmax()
most_active_day_freq = most_ads_by_day.max()

print(f'Hari dengan iklan terbanyak: {most_active_day} (sebanyak {most_active_day_freq} pengguna)')


Hari dengan iklan terbanyak: Friday (sebanyak 92608 pengguna)


**Interpretasi**

Jumat merupakan hari dengan paparan iklan tertinggi, menunjukkan bahwa:

- Kampanye iklan paling banyak tampil di hari Jumat.

- Pengguna paling sering berinteraksi dengan iklan pada hari ini.

Bisa jadi karena pengguna lebih aktif menjelang akhir pekan, atau strategi penayangan iklan memang difokuskan pada Jumat.

**Insight Guardrail**

Potensi oversaturation: Jika terlalu banyak iklan ditayangkan di hari yang sama (Jumat), hal ini dapat menimbulkan kejenuhan pengguna, penurunan efektivitas iklan, atau bahkan peningkatan skip rate / bounce.

Risiko evaluasi bias: Keberhasilan kampanye bisa tampak tinggi hanya karena terkonsentrasi di hari tertentu, bukan karena kualitas iklan itu sendiri.

**Rekomendasi**

Sebarkan distribusi iklan lebih merata ke hari-hari lain, terutama Selasa–Kamis, yang biasanya juga hari aktif secara digital.

Lakukan analisis performa kampanye berdasarkan hari, misalnya dengan mengukur:

- Conversion rate tiap hari

- Click-through rate (CTR)

- Engagement rate